# Advanced Programme in Deep Learning (Foundations and Applications)
## A Program by IISc and TalentSprint
### Assignment : Natural Language Processing - II (Bidirectional GRU for Sentence Classification)

### Learning Objectives:

At the end of the experiment, you will be able to:

*  generate vector representation of words in the data using Glove embeddings
*  implement the multi-layer bidirectional GRU (Gated
Recurrent Unit) for solving the sentence classification problem

### Setup Steps:

In [1]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "2240589" #@param {type:"string"}

In [2]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "9886499911" #@param {type:"string"}

In [3]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython
import warnings
warnings.filterwarnings("ignore")

ipython = get_ipython()

notebook= "M3_AST_17_Bidirectional_GRU_for_Sentence_Classification_C" #name of the notebook

def setup():
    ipython.magic("sx wget -qq https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/glove.6B.zip")
    ipython.magic("sx unzip glove.6B.zip")
    ipython.magic("sx wget https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/rt-polarity.zip")
    ipython.magic("sx unzip rt-polarity.zip")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer1() and getAnswer2() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer1" : Answer1, "answer2" : Answer2, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://dlfa-iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer1():
  try:
    if not Answer1:
      raise NameError
    else:
      return Answer1
  except NameError:
    print ("Please answer Question 1")
    return None

def getAnswer2():
  try:
    if not Answer2:
      raise NameError
    else:
      return Answer2
  except NameError:
    print ("Please answer Question 2")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


### Dataset Description

The **sentence polarity dataset v1.0** contains two data files which are:
  * **rt-polarity.pos**: It contains 5331 positive examples
  * **rt-polarity.neg**: It contains 5331 negative examples

Each line in these two files corresponds to a single snippet (usually
containing roughly one single sentence) that includes the review of a movie.

**Note:** Here is the source [link](https://www.cs.cornell.edu/people/pabo/movie-review-data/) to the Movie  dataset





### Introduction

The aim of this assignment is to study the use of multi-layer bidirectional GRU (Gated
Recurrent Unit) for solving the sentence classification problem. You will study the effect of adding
layers of BiGRU units on the test set performance of the model. The dataset used will be the same
as that was used in the first assignment.

### Importing the libraries and packages

In [4]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from gensim.utils import simple_preprocess

import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder

from keras.layers import Input, Embedding, Dense, Bidirectional, Dropout, GRU
from keras.models import Sequential   # the model

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Loading the data

In [5]:
# Read the positive and negative files and split the sentences into a list
with open('rt-polarity.neg',"r") as data_neg:
  data_neg_set = data_neg.read().splitlines()

with open('rt-polarity.pos',"r") as data_pos:
  data_pos_set = data_pos.read().splitlines()

In [6]:
# Length of the positive and negative reviews
len(data_neg_set), len(data_pos_set)

(5331, 5331)

In [7]:
# Loading the negative reviews
data_neg_set = pd.DataFrame(data_neg_set, columns=["Review"])

# Loading the positive reviews
data_pos_set = pd.DataFrame(data_pos_set, columns=["Review"])

In [8]:
# Print the first five rows of the positive examples
data_pos_set.head()

,Review
0,the rock is destined to be the 21st century's ...
1,"the gorgeously elaborate continuation of "" the..."
2,effective but too-tepid biopic
3,if you sometimes like to go to the movies to h...
4,"emerges as something rare , an issue movie tha..."


In [9]:
# Print the first five rows of the negative examples
data_neg_set.head()

,Review
0,"simplistic , silly and tedious ."
1,"it's so laddish and juvenile , only teenage bo..."
2,exploitative and largely devoid of the depth o...
3,[garbus] discards the potential for pathologic...
4,a visually flashy but narratively opaque and e...


#### Giving the labels to the data

Let us give the labels as positive and negative for the sentences present in the two files.

In [10]:
data_neg_set['Polarity'] = 'Negative'
data_pos_set['Polarity'] = 'Positive'

Let us have a glance at few of the values present in the data with negative and positive reviews that we have labeled in the previous step.

In [11]:
data_neg_set.head()

,Review,Polarity
0,"simplistic , silly and tedious .",Negative
1,"it's so laddish and juvenile , only teenage bo...",Negative
2,exploitative and largely devoid of the depth o...,Negative
3,[garbus] discards the potential for pathologic...,Negative
4,a visually flashy but narratively opaque and e...,Negative


#### Combining the positive and negative data

Now, we have to work on the combined data containing the positive and negative reviews, so, let us concatenate both the dataframes.

In [12]:
dataframes = [data_neg_set, data_pos_set]
rt_polarity_data = pd.concat(dataframes)
rt_polarity_data

,Review,Polarity
0,"simplistic , silly and tedious .",Negative
1,"it's so laddish and juvenile , only teenage bo...",Negative
2,exploitative and largely devoid of the depth o...,Negative
3,[garbus] discards the potential for pathologic...,Negative
4,a visually flashy but narratively opaque and e...,Negative
...,...,...
5326,both exuberantly romantic and serenely melanch...,Positive
5327,mazel tov to a film about a family's joyous li...,Positive
5328,standing in the shadows of motown is the best ...,Positive
5329,it's nice to see piscopo again after all these...,Positive



From above we can see that due to concatenation the last row index is 5330 that we can see below but that should be 10661.

Therefore, we will reset the index so that last row has index of 10661.

In [13]:
rt_polarity_data.reset_index(inplace=True, drop=True)

In [14]:
rt_polarity_data

,Review,Polarity
0,"simplistic , silly and tedious .",Negative
1,"it's so laddish and juvenile , only teenage bo...",Negative
2,exploitative and largely devoid of the depth o...,Negative
3,[garbus] discards the potential for pathologic...,Negative
4,a visually flashy but narratively opaque and e...,Negative
...,...,...
10657,both exuberantly romantic and serenely melanch...,Positive
10658,mazel tov to a film about a family's joyous li...,Positive
10659,standing in the shadows of motown is the best ...,Positive
10660,it's nice to see piscopo again after all these...,Positive


When you combine the negative and positive examples, it is a good idea to shuffle the examples so that the negative and positive examples are spread throughout. If we do not shuffle it, then, it may happen that in some mini-batches, examples from only one class(positive or negative) will be present. Therefore, it is better to avoid such scenarios.


In [15]:
rt_polarity_data = shuffle(rt_polarity_data)

In [16]:
rt_polarity_data

,Review,Polarity
10405,the movie worked for me right up to the final ...,Positive
1583,"excessive , profane , packed with cartoonish v...",Negative
6250,a comprehensive and provocative film -- one th...,Positive
7364,as the movie traces mr . brown's athletic expl...,Positive
6861,a giggle-inducing comedy with snappy dialogue ...,Positive
...,...,...
6218,the reason this picture works better than its ...,Positive
7414,"gorgeous scenes , masterful performances , but...",Positive
973,full frontal had no effect and elicited no sym...,Negative
7093,"a movie that both thrills the eye and , in its...",Positive


Let us check the value counts of negative and positive reviews.

In [17]:
rt_polarity_data['Polarity'].value_counts()

Positive    5331
Negative    5331
Name: Polarity, dtype: int64

Checking whether there are any null values present in the data.

In [18]:
rt_polarity_data.isnull().values.any()

False

### Label Encoding

In [19]:
# Converting the labels from categorical to numerical
le = LabelEncoder()
rt_polarity_data['Polarity'] = le.fit_transform(rt_polarity_data['Polarity'])
rt_polarity_data.head()

,Review,Polarity
10405,the movie worked for me right up to the final ...,1
1583,"excessive , profane , packed with cartoonish v...",0
6250,a comprehensive and provocative film -- one th...,1
7364,as the movie traces mr . brown's athletic expl...,1
6861,a giggle-inducing comedy with snappy dialogue ...,1


### Data Preprocessing


We can preprocess the text using gensim package. Gensim provides function **simple_preprocess** for more effective preprocessing of the corpus. In such kind of preprocessing, we can convert a document into a list of lowercase tokens. We can also ignore tokens that are too short or too long.

**Note:** Refer to the following [link](https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess) for gensim `simple_preprocess` method

In [20]:
rt_polarity_data['Review'] = rt_polarity_data['Review'].apply(lambda x:simple_preprocess(x, max_len=30))

In [21]:
# Remove stop words
stop_words = set(stopwords.words('english'))

rt_polarity_data['Review'] = rt_polarity_data['Review'].apply(lambda x: [w for w in x if not w in stop_words])

In [22]:
rt_polarity_data.head()

,Review,Polarity
10405,"[movie, worked, right, final, scene, caved]",1
1583,"[excessive, profane, packed, cartoonish, viole...",0
6250,"[comprehensive, provocative, film, one, pushes...",1
7364,"[movie, traces, mr, brown, athletic, exploits,...",1
6861,"[giggle, inducing, comedy, snappy, dialogue, w...",1


### Hyperparameters

In [23]:
# Hyperparameters
MAX_SENT_LEN = 30   # Number of words to consider from each review
MAX_VOCAB_SIZE = 20000  # Max vocabulary size
BATCH_SIZE = 32
N_EPOCHS = 15

### Tokenize and Pad sequences

A Neural Network only accepts numeric data, so we need to encode the reviews. Here use keras.Tokenizer() to encode the reviews into integers, where each unique word is automatically indexed (using `fit_on_texts` method) calculates the frequency of each word in our corpus/messages.

`texts_to_sequences` method finally converts our array of sequences of strings to list of sequences of integers (most frequent word is assigned 1 and so on).

Each reviews has a different length, so we need to add padding (by adding 0) or truncating the words to the same length (in this case, it is the mean of all reviews length) using `keras.preprocessing.sequence.pad_sequences.`

`post`, pad or truncate the words in the back of a sentence
`pre`, pad or truncate the words in front of a sentence

Each word is assigned an integer and that integer is placed in a list.


For example if we have a sentence “How text to sequence and padding works”. Each word is assigned a number. We suppose how = 1, text = 2, to = 3, sequence = 4, and = 5, padding = 6, works = 7. After texts_to_sequences is called our sentence will look like [1, 2, 3, 4, 5, 6, 7 ]. Now for suppose our MAX_SEQUENCE_LENGTH = 10. After padding our sentence will look like `pre` = [0, 0, 0, 1, 2, 3, 4, 5, 6, 7 ], `post` = [1, 2, 3, 4, 5, 6, 7, 0, 0, 0]

In [24]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts([' '.join(seq[:MAX_SENT_LEN]) for seq in rt_polarity_data['Review']])

print("Number of words in vocabulary:", len(tokenizer.word_index))

Number of words in vocabulary: 18007


In [25]:
# Convert the sequence of words to sequnce of indices
X = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in rt_polarity_data['Review']])
X = pad_sequences(X, maxlen=MAX_SENT_LEN, padding='post', truncating='post')

y = rt_polarity_data['Polarity']

### Splitting the data into train and test sets

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42, train_size=10000)

In [27]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((10000, 30), (662, 30), (10000,), (662,))

### Load the GloVe word embeddings

**What is GloVe?**

GloVe stands for global vectors for word representation. It is an unsupervised learning algorithm developed by Stanford for generating word embeddings by aggregating global word-word co-occurrence matrix from a corpus. Word embeddings are basically a form of word representation that bridges the human understanding of language to that of a machine. Meaning that two similar words are represented by almost similar vectors that are very closely placed in a vector space. These are essential for solving most Natural language processing problems.The resulting embeddings show interesting linear substructures of the word in vector space.

Thus when using word embeddings, all individual words are represented as real-valued vectors in a predefined vector space. Each word is mapped to one vector and the vector values are learned in a way that resembles a neural network.

Now, let us load the 300-dimensional GloVe embeddings.

In [28]:
embeddings_index = {}
# Loading the 300-dimensional vector of the model
f = open('/content/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [29]:
# Adding 1 because of reversed 0 index
words_not_found = []
vocab_size = len(tokenizer.word_index) + 1
print('Loaded %s word vectors.' % len(embeddings_index))

embedding_dim = 300

# Create a weight matrix for words in the training data
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i >= vocab_size:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
                embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)

Loaded 400000 word vectors.


In [30]:
print(tokenizer.word_index)

{'film': 1, 'movie': 2, 'one': 3, 'like': 4, 'story': 5, 'good': 6, 'much': 7, 'even': 8, 'comedy': 9, 'time': 10, 'characters': 11, 'well': 12, 'director': 13, 'little': 14, 'way': 15, 'funny': 16, 'make': 17, 'enough': 18, 'never': 19, 'life': 20, 'makes': 21, 'bad': 22, 'may': 23, 'best': 24, 'us': 25, 'work': 26, 'love': 27, 'would': 28, 'made': 29, 'new': 30, 'movies': 31, 'something': 32, 'could': 33, 'action': 34, 'really': 35, 'drama': 36, 'two': 37, 'see': 38, 'plot': 39, 'performances': 40, 'long': 41, 'many': 42, 'still': 43, 'films': 44, 'look': 45, 'old': 46, 'every': 47, 'big': 48, 'people': 49, 'nothing': 50, 'first': 51, 'better': 52, 'get': 53, 'fun': 54, 'without': 55, 'great': 56, 'character': 57, 'might': 58, 'though': 59, 'ever': 60, 'audience': 61, 'world': 62, 'also': 63, 'real': 64, 'another': 65, 'cast': 66, 'kind': 67, 'humor': 68, 'self': 69, 'sense': 70, 'script': 71, 'year': 72, 'feel': 73, 'less': 74, 'often': 75, 'performance': 76, 'far': 77, 'thing': 78,

In [31]:
print(len(tokenizer.word_index))

18007


### Define the Bi-directional GRU model



### LSTM vs GRU
<center>
<img src="https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/GRU.png" width=700px, height=500/>
</center>
<br><br>

Simple RNNs have a very short memory, due to the issue of vanishing gradients. More complicated cell architectures try to solve the short memory problem. The most famous one is probably the Long Short Term Memory (LSTM) cell:


It uses a gated cell architecture to update and forget information selectively in the network memory (cell and hidden states). The Gated Recurrent Units (GRU) have a slightly simpler architecture (and only one hidden state). GRUs are usually faster than LSTMs, while still often have competitive performances for many applications.

### GRU - The subtle differences

* The **update gate** acts similar to the **forget gate** and **input gate** of an LSTM

* The **update gate** decides how much of the past information (from previous time steps) needs to be passed along to the future.

* The **reset gate** decides how much of the past information to forget

* Some tensor ops and speedier to train than LSTMs

<center>
<img src="https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/GRU_Subtle_difference.png" width=700px, height=500/>
</center>
<br><br>

### The need for Bi-directional GRUs

* Bi-directional GRUs are just putting two independent GRUs together

* The input sequence is fed in forward order for one GRU, and reverse order for the other

* The otputs of the two networks are usually concatenated at each time step

* Preserving information from both past and future helps understand context better

<center>
<img src="https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/Bi-GRU.jpg" width=700px, height=500/>
</center>


A bidirectional GRU consists of forward layer and a backward layer. The input sequence is fed to the forward layer in the regular way, while in the backward layer the input is processed in the reverse order, starting from the last word, then proceed to the next to last word and so on up to to first word.

The hidden states are then concatenated for each token generating an intermediate representation sequence. Hence, for each intermediate representation the information from the sequence before and after the respective token are taken into account. That means for each iteration step the network has access to the complete document and can deduce the right label from that information.

In [32]:
# Build a sequential model by stacking neural net units
model = Sequential()
embedding_layer = Embedding(vocab_size,
                            embedding_dim,
                            weights = [embedding_matrix],
                            input_length = MAX_SENT_LEN,
                            trainable=False)
model.add(embedding_layer)
model.add(Bidirectional(GRU(128, return_sequences=True, dropout=0.50, name='first_gru_layer')))
model.add(Dropout(0.5))
model.add(Bidirectional(GRU(64, name='second_gru_layer')))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid', name='output_layer'))

In [33]:
print('Summary of the built model...')
model.summary()

Summary of the built model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 300)           5402400   
                                                                 
 bidirectional (Bidirection  (None, 30, 256)           330240    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 30, 256)           0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               123648    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)            

### Compile and train the model

In [34]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [35]:
model.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=N_EPOCHS,
          validation_data=(X_test, y_test))

Epoch 1/15
313/313 [==============================] - 20s 18ms/step - loss: 0.6131 - accuracy: 0.6583 - val_loss: 0.5292 - val_accuracy: 0.7341
Epoch 2/15
313/313 [==============================] - 4s 11ms/step - loss: 0.5444 - accuracy: 0.7331 - val_loss: 0.5068 - val_accuracy: 0.7613
Epoch 3/15
313/313 [==============================] - 4s 11ms/step - loss: 0.5177 - accuracy: 0.7463 - val_loss: 0.4872 - val_accuracy: 0.7840
Epoch 4/15
313/313 [==============================] - 4s 13ms/step - loss: 0.5032 - accuracy: 0.7609 - val_loss: 0.4803 - val_accuracy: 0.7810
Epoch 5/15
313/313 [==============================] - 3s 11ms/step - loss: 0.4880 - accuracy: 0.7699 - val_loss: 0.4763 - val_accuracy: 0.7810
Epoch 6/15
313/313 [==============================] - 4s 12ms/step - loss: 0.4706 - accuracy: 0.7775 - val_loss: 0.4736 - val_accuracy: 0.7779
Epoch 7/15
313/313 [==============================] - 5s 16ms/step - loss: 0.4576 - accuracy: 0.7824 - val_loss: 0.4768 - val_accuracy: 0.785

### Evaluate the model

In [36]:
print('Testing...')
model.evaluate(X_test, y_test)

Testing...
21/21 [==============================] - 0s 7ms/step - loss: 0.5894 - accuracy: 0.7719


[0.5893772840499878, 0.7719033360481262]

In [37]:
# model predictions on the test data
preds = model.predict(X_test)

21/21 [==============================] - 1s 6ms/step


In [38]:
preds.shape

(662, 1)

In [39]:
# Get the text sequences for the preprocessed movie reviews
reviews_list_idx = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in rt_polarity_data['Review']])

In [40]:
print(reviews_list_idx[1])

[5407, 4445, 1055, 3206, 453, 199, 3747, 11]


In [41]:
# Function to get the predictions on the movie reviews using GRU model
def add_score_predictions(data, reviews_list_idx):

  # Pad the sequences of the data
  reviews_list_idx = pad_sequences(reviews_list_idx, maxlen=MAX_SENT_LEN, padding='post', truncating='post')

  # Get the predictons by using GRU model
  review_preds = model.predict(reviews_list_idx)

  # Add the predictions to the movie reviews data
  rt_polarity_data['polarity score'] = review_preds

  # Set the threshold for the predictions
  pred_sentiment = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative', review_preds)))

  # Add the sentiment predictions to the movie reviews
  rt_polarity_data['predicted polarity'] = pred_sentiment

  return rt_polarity_data

In [42]:
# Call the above function to get the sentiment score and the predicted sentiment
data = add_score_predictions(rt_polarity_data, reviews_list_idx)

334/334 [==============================] - 1s 4ms/step


In [43]:
# Display the data
data[:20]

,Review,Polarity,polarity score,predicted polarity
10405,"[movie, worked, right, final, scene, caved]",1,0.461928,negative
1583,"[excessive, profane, packed, cartoonish, viole...",0,0.003545,negative
6250,"[comprehensive, provocative, film, one, pushes...",1,0.998912,positive
7364,"[movie, traces, mr, brown, athletic, exploits,...",1,0.996537,positive
6861,"[giggle, inducing, comedy, snappy, dialogue, w...",1,0.992384,positive
8823,"[intimate, spectacular, carried, less, wow, fa...",1,0.999274,positive
8412,"[short, story, quaint, touchingly, mending, ch...",1,0.991057,positive
8870,"[saw, enjoyed]",1,0.909851,positive
3300,"[moments, real, pleasure, found, sara, sugarma...",0,0.038219,negative
1015,"[movie, lumbering, load, hokum, least, watchable]",0,0.081917,negative


### Please answer the questions below to complete the experiment:




**Consider the following statements and answer Q1:**


A. The GRU controls the past information by having two gates: an update gate and reset gate, the update gate determines how much of the past knowledge needs to be passed along into the future and the reset gate is used to decide how much of the past information to forget.

B. GRUs handles the vanishing gradient problem in recurrent neural networks very efficiently.

C. GRUs cannot handle the vanishing gradient problem in recurrent neural networks very efficiently.

D. GRUs solve the problem of exploding gradients using gradient clipping in which a defined threshold value is set on the gradients, which means that even if a gradient increases beyond the predefined value during training, its value will still be limited to the set threshold.



In [44]:
#@title Q1. Which of the above statement(s) is/are True? { run: "auto", form-width: "500px", display-mode: "form" }
Answer1 = "A, B and D" #@param ["", "Only A and B", "A, B and D", "Only C", "only A", "only D"]

**Consider the following statements and answer Q2:**


A. Bidirectional GRU, or BiGRU, is a sequence processing model, consisting of two GRUs, one taking the input in a forward direction, and the other in a backwards direction.

B. Bidirectional GRUs are  a type of bidirectional recurrent neural networks with only forget gates.

C. GRUs use less training parameters and therefore use less memory and execute faster and generally have a better performance than LSTMs when dealing with smaller datasets.

D. For GRU bidirectional layer output, we take the sum of outputs of both forward and backward direction layers




In [45]:
#@title Q2. Which of the above statement(s) is/are True? { run: "auto", form-width: "500px", display-mode: "form" }
Answer2 = "A and C" #@param ["", "A and B","A and C","B and D","C and D", "only A", "only C"]

In [46]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good and Challenging for me" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [47]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "na" #@param {type:"string"}


In [48]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["","Yes", "No"]


In [49]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [50]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [51]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")

Your submission is successful.
Ref Id: 1570
Date of submission:  23 Sep 2023
Time of submission:  15:17:55
View your submissions: https://dlfa-iisc.talentsprint.com/notebook_submissions
